In [1]:
!export HF_DATASETS_CACHE="/scratch/pt2295/cache/"

In [2]:
!export CUDA_VISIBLE_DEVICES=0,1,2,3

In [3]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/scratch/pt2295/cache/'

In [4]:
import json
import random
import re



file_path = "/scratch/pt2295/LlvmProject/automatic_prompt_engineer/data/bigbench-ii/epistemic_reasoning/task.json"

with open(file_path, 'r') as json_file:
    
    data_orig = json.load(json_file)

In [5]:
data=data_orig['examples']

In [6]:
len(data_orig['examples'])

2000

In [7]:
random.shuffle(data)

# Define the proportions for train, test, and validation sets
total_samples = len(data)
train_ratio = 0.7
test_ratio = 0.15
validation_ratio = 0.15

# Calculate the sizes of each set
train_size = int(total_samples * train_ratio)
test_size = int(total_samples * test_ratio)
validation_size = int(total_samples * validation_ratio)

# Split the data into sets
train_data = data[:train_size]
test_data = data[train_size:train_size + test_size]
validation_data = data[train_size + test_size:]

In [8]:
len(train_data)

1400

In [9]:
train_data=train_data[0:100]

# Algo

In [10]:
task_description=data_orig['description']

In [11]:
task_type='entailment'

In [12]:
task_prefix=data_orig['task_prefix']
if task_type=='hyperbaton':
    task_prefix=task_prefix.strip()
    task_prefix+=" Choose only from the following options: 'a' or 'b'.\n"

In [13]:
def preprocess(text):
    return

In [14]:
def extract_answer(length,line,task_type='entailment'):
    line=line[length:]
    pattern = r'<Ans>(.*?)</Ans>|Ans:\s*([\w-]+)'
    matches = re.findall(pattern, line)
    if len(matches)==0:
        return -1
#     print("WHAT",matches)
    if task_type=='entailment':
        if matches =='entailment' or matches=='Entailment' or "non-entail" not in matches:
            return 1
    elif task_type=='hyperbaton':
        print(matches[0][0])
        if matches[0][0]=='a' or matches[0][0]=='A':
            return 1
    return 0
  

In [15]:
from transformers import AutoTokenizer,AutoModelForCausalLM
import transformers
import torch

model_name = "meta-llama/Llama-2-13b-chat-hf"
custom_cache_dir = "/scratch/pt2295/cache/"
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=custom_cache_dir,use_fast=True)

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model_name,
#     torch_dtype=torch.float16,
#     device_map="auto",
#     tokenizer=tokenizer
    
# )
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True,cache_dir=custom_cache_dir)




/ext3/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:37<00:00, 12.63s/it]


In [16]:
def get_answer_llm(user_prompt):
# user_prompt="Determine whether one sentence entails the next. Given input: Premise: William suspects that Isabella learns that two men are in blue life jackets and sitting on innertubes with 'Crusher' written on them in the middle of a body of water. Hypothesis: Isabella learns that two men are in blue life jackets and sitting on innertubes with 'Crusher' written on them in the middle of a body of water.Identify the relation between the following premises and hypotheses, choosing from the options 'entailment' or 'non-entailment'. Put your answer within tag <Ans> and </Ans>"
    sys_prompt="You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."
    prompt=f"<s>[INST] {user_prompt} [/INST]"
    
#     sequences = pipeline(
#     prompt,
#     max_new_tokens=200
#     )
#     return sequences[0]["generated_text"][len(prompt):]
    
   
    model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    output = model.generate(**model_inputs)

#     print(tokenizer.decode(output[0], skip_special_tokens=True))
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [17]:
# model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True)
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, use_auth_token=access_token)
# model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda:0")

# output = model.generate(**model_inputs)

# print(tokenizer.decode(output[0], skip_special_tokens=True))

In [18]:
# for seq in sequences:
#     print(f"Result: {seq['generated_text']}")
#     print()
    

In [19]:
def run_inference_step1(task_type='entailment'):
    wrong_ans_indices=[]
    not_got=[]
    for i in range(len(train_data)):
        
        prompt_t= task_description+'\n'+'Given input: '+train_data[i]['input']+'\n'+task_prefix+'Put your one-word answer choosing from the previously stated two options within tag <Ans> and </Ans>. The one word answer should be necassarily between the tags.'
        op=get_answer_llm(prompt_t)
        output_str=op#"".join([i['generated_text'] for i in op])

#         print(prompt_t)
        ans=extract_answer(len(prompt_t),output_str,task_type)
        if ans==-1:
            not_got.append(i)
            continue
        ground_truth=1
        print("$$$$$",ans)
        if task_type=='entailment':
            if train_data[i]['target_scores']['non-entailment']==1:
                ground_truth=0
        elif task_type=='hyperbaton':
            if train_data[i]['target_scores']['b']==1:
                ground_truth=0
        if (ans!=ground_truth):
            wrong_ans_indices.append(i)
        
    
        
    
    return wrong_ans_indices

In [20]:
wrong_answers=run_inference_step1(task_type)

/ext3/miniconda3/lib/python3.11/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1
$$$$$ 1


In [ ]:
# answer not only within tags, difficult to extract answer -> solved by improving prompt
#70b is good but v slow
#maybe use pipeline with datasets or use more gpus

In [32]:
def extract_hint(length,line):
    line=line[length:]
    pattern = r'<hint>(.*?)</hint>'
    matches = re.findall(pattern, line)
    if len(matches)==0:
        return ""
    return matches[0]

In [41]:
(wrong_answers)

[0,
 1,
 3,
 5,
 6,
 7,
 8,
 9,
 10,
 13,
 14,
 15,
 16,
 17,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 39,
 41,
 43,
 44,
 45,
 46,
 49,
 50,
 51,
 52,
 54,
 55,
 57,
 58,
 59,
 60,
 62,
 63,
 64,
 66,
 69,
 72,
 73,
 75,
 76,
 77,
 81,
 84,
 85,
 88,
 89,
 90,
 91,
 93,
 97,
 99]

In [34]:
# len(not_got_append)

In [45]:
def get_hints_residual_step2():
    hints_wrong_ans=[]
    for idx in wrong_answers:
        if task_type=='entailment':
            if train_data[idx]['target_scores']['entailment']==1:
                ans='entailment'
            else:
                ans='non-entailment'
        prompt_h='Given following task:'+task_description+'\n'+'Given input: '+train_data[idx]['input']+'\n'+'And its expected output: '+ans+'\n'+'List the reason or hint why its with this expected output within tag <hint> and </hint>. The hint or explaination should be necassarily between the tags.'
       
        op=get_answer_llm(prompt_h)
        hint=extract_hint(len(prompt_h),op)
       
        if hint!='':
            hints_wrong_ans.append((idx,hint))
        
    return hints_wrong_ans
    

In [46]:
hints_wrong=get_hints_residual_step2()

In [47]:
hints_wrong

[(0,
  "Think about the difference between knowledge and beliefs. James's beliefs about Taylor's knowledge are not necessarily the same as Taylor's actual knowledge."),
 (1,
  'The premise and hypothesis are both statements about the same thing - a speaking woman at a podium with a specific phrase written on it. There is no new information or difference between the two statements to establish an entailment relationship.'),
 (3,
  'Note that the premise and hypothesis are both statements about beliefs and suspicions, but they do not provide enough information to determine a logical connection between the two.'),
 (5,
  'Note that the hypothesis is not a direct restatement of the premise, but rather a rearrangement of the words. This is a common technique used in logic puzzles to test the ability to recognize non-entailment.'),
 (6,
  'Note that the hypothesis is not providing any new information or perspective that would lead to the conclusion. The premise and hypothesis are essentially